# Creating the Channel Funding Transaction

In this section, we’ll create a Lightning channel funding transaction from scratch using Python. We’ll walk through each part of the transaction—how it’s constructed, signed, and how peers exchange messages to share the necessary information to make it happen. We'll test everything using Bitcoin Core in regtest mode.


## Setup

For this exercise we'll need Bitcoin Core. This notebook has been tested with v29.0

Below, set the paths for:

    The bitcoin core functional test framework directory.
    The directory containing taproot-lightning-channels-workshop.

You'll need to edit these next two lines for your local setup.


In [1]:
path_to_bitcoin_functional_test = "/home/pins-dev/Projects/bitcoin/build/test/functional"
path_to_taproot_workshop = "/home/pins-dev/Projects/Taproot-Lightning-Channels-Workshop"


### Setup bitcoin core test framework

Start up regtest mode, delete any regtest network history so we are starting from scratch.

In [2]:
import sys

# Add the functional test framework to our PATH
sys.path.insert(0, path_to_bitcoin_functional_test)
from test_framework.test_shell import TestShell

# Add the bitcoin-tx-tutorial functions to our PATH
sys.path.insert(0, path_to_taproot_workshop)
from functions import *
from functions.bip_0340_reference import *
from functions.test_framework.musig import generate_musig_key
from functions.test_framework.key import generate_bip340_key_pair 
from functions.test_framework.script import tagged_hash
from functions.test_framework.address import program_to_witness


import json

# Setup our regtest environment
test = TestShell().setup(
    num_nodes=1, 
    setup_clean_chain=True
)

node = test.nodes[0]

# Create a new wallet and address to send mining rewards so we can fund our transactions
node.createwallet(wallet_name='mywallet')
address = node.getnewaddress()

# Generate 101 blocks so that the first block subsidy reaches maturity
result = node.generatetoaddress(nblocks=101, address=address, called_by_framework=True)

# Check that we were able to mine 101 blocks
assert(node.getblockcount() == 101)

2025-08-27T01:18:59.247000Z TestFramework (INFO): PRNG seed is: 4449761118161828305
2025-08-27T01:18:59.249000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_01ey37wi


### Setup Alice and Bob Node Wallet

Create and fund Taproot addresses for Alice and Bob so they have funds to open a simple Taproot channel. The UTXO created for Alice will be the input of the Channel Funding Transaction.

In [3]:
# Create a new address to alice so she can sign the funding transaction
alice_wallet_ctx = generate_taproot_ctx()
alice_funding_address = generate_taproot_address(alice_wallet_ctx, index=0, change=False)
print(f"Alice funding address created: {alice_funding_address}")

# Create a new address to bob so he can sign the funding transaction
bob_wallet_ctx = generate_taproot_ctx()
bob_funding_address = generate_taproot_address(bob_wallet_ctx, index=0, change=False)
print(f"Bob funding address created: {bob_funding_address}")

# Send 1 BTC to Alice and Bob
txid = node.sendmany("", {alice_funding_address: 1, bob_funding_address: 1})
result = node.generatetoaddress(nblocks=1, address=address, called_by_framework=True)

Alice funding address created: bcrt1px744uu3vtpycs7xcuvznc8rfdrg3rldp74y56088lsyt5ju4xujqzelpl7
Bob funding address created: bcrt1p5p869d8k3n59ulwsp8rk26hscvqflhlx224eqwrgylscmuf4grasq9y0m9


## Channel Establishment

The "[Extention Bolt](https://github.com/lightning/bolts/pull/995)" for simple taproot channels (work in progress) defines a pathway to create a channel. The messages defined in [BOLT 2](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1) are used to exchange the information needed to create the Channel Funding Transaction, as shown below.


    +-----------+                              +---------+
    |           |--(1)---  open_channel  ----->|         |
    |           |<-(2)--  accept_channel  -----|         |
    |           |                              |         |
    |   Alice   |--(3)--  funding_created  --->|   Bob   |
    |           |<-(4)--  funding_signed  -----|         |
    |           |                              |         |
    |           |--(5)---  channel_ready  ---->|         |
    |           |<-(6)---  channel_ready  -----|         |
    +-----------+                              +---------+


* (1) open_channel - Alice share its "next_local_nonce"

* (2) accept_channel - Bob share its "next_local_nonce"

* (3) funding_created - Alice samples a fresh nonce, combines it with next_local_nonce received to create a MuSig2 partial signature, then shares partial_signature_with_nonce.

* (4) funding_signed - Bob samples a fresh nonce, combines it with next_local_nonce received to create a MuSig2 partial signature, then shares partial_signature_with_nonce.

* (5) channel_ready - Alice share its "next_local_nonce"

* (6) channel_ready - Bob share its "next_local_nonce"


All the nonces are generated using "NonceGen" algorithm defined in [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki) to ensure it generates nonces in a safe manner.

The pubkeys are sorted using the "KeySort" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

To compute the aggregated musig2 public key from the sorted funding_pubkeys use the "KeyAgg" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

To construct a musig2 partial signature for the sender's remote commitment use the "Sign" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

### The Channel Funding Transaction

Alice has a 1 BTC UTXO that will serve as the funding transaction input. We’ll now create two outputs: a change output back to Alice and the channel-funding output.

In [4]:
alice_change_address = generate_taproot_address(alice_wallet_ctx, index=0, change=True)
print(f"Alice change address created: {alice_change_address}")

Alice change address created: bcrt1pxvzpuxn4hpkvue945rsrjya8dkep79pztw99cxvmd297h7q3hglqedvtks


The diagram below illustrates how a Taproot channel funding output is constructed.

    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^
                   |   +----------+
                    ---| Pagg + T |
                       +----------+
                         ^      ^
                         |      |  +-----------+
                         |       --| T = t * G |<---------
                         |         +-----------+          |
                         |                                |
           +---------------------------------------+    +---+   +-------------------------------+
           |      generate_musig_key(Pa, Pb)       |    | t | = | TaggedHash ("Taptweak", Pagg) |
           +---------------------------------------+    +---+   +-------------------------------+

In [5]:
# Create a new Alice key pair for channel funding multisig output
privkey_alice_musig, pubkey_alice_musig = generate_bip340_key_pair()
print(f"Alice MuSig pubkey: {pubkey_alice_musig.get_bytes().hex()}")
print(f"Alice MuSig privkey: {privkey_alice_musig.get_bytes().hex()}")

# Create a new Bob key pair for channel funding multisig output
privkey_bob_musig, pubkey_bob_musig = generate_bip340_key_pair()
print(f"Bob MuSig pubkey: {pubkey_bob_musig.get_bytes().hex()}")
print(f"Bob MuSig privkey: {privkey_bob_musig.get_bytes().hex()}")

# Generate a 2-of-2 aggregate MuSig key using the pubkeys form Alice and Bob
c_map, agg_pubkey = generate_musig_key([pubkey_alice_musig, pubkey_bob_musig])

# Multiply individual keys with challenges
privalice_c = c_map[pubkey_alice_musig] * privkey_alice_musig
privbob_c = c_map[pubkey_bob_musig] * privkey_bob_musig
pubalice_c = c_map[pubkey_alice_musig] * pubkey_alice_musig
pubbob_c = c_map[pubkey_bob_musig] * pubkey_bob_musig

if agg_pubkey.get_y()%2 != 0:
    privalice_c.negate()
    privbob_c.negate()
    pubalice_c.negate()
    pubbob_c.negate()
    agg_pubkey.negate()

print(f"Aggregate musig pubkey: {agg_pubkey.get_bytes().hex()}")

# Tweak musig public key
# Method: ECPubKey.tweak_add()
tweak = tagged_hash("TapTweak", agg_pubkey.get_bytes())
agg_pubkey_tweaked = agg_pubkey.tweak_add(tweak)
print(f"Tweaked aggregate musig pubkey: {agg_pubkey_tweaked.get_bytes().hex()}")

agg_pubkey_tweaked_b = agg_pubkey_tweaked.get_bytes()

# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
channel_funding_musig_address = program_to_witness(0x01, agg_pubkey_tweaked_b)

print(f"Channel funding musig output address: {address}")

Alice MuSig pubkey: ccdf3671eee1da997e8bec544c772a673c5cf4b30a63eaec21e9dc16825d80ce
Alice MuSig privkey: c71ac1784968717dcfa3e0efa5ba49403ff86db68a4f16a08f4374d73442243d
Bob MuSig pubkey: cd290998680f4d835050826f05724ee29980fd8f1d0ea4ee17e623821609a7b4
Bob MuSig privkey: 84a95722e7e5e970fd2772148eca9e60a4806f71d8594d43dceca8840c88789e
Aggregate musig pubkey: e825c3107fc36f8ea9045466cf65916880f5d522dd01560bc4f4d0420a9c08d1
Tweaked aggregate musig pubkey: 9aecad700bf8b9783a0d652e9c7ebd1a6a08be9a169402dd81a67f35a3499dd8
Channel funding musig output address: bcrt1qyxxphcs2f22szckqs6y5r844xd28necqg08acx
